In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
# Read bench data

# To fix the probem with the way output data is structured after multiple runs of onell

#bench_file_name = 'output_5'
bench_file_name = 'output_6_1'

with open(bench_file_name + '.json') as raw_file:
    with open(bench_file_name + '_fixed.json', 'wt') as json_file:
        for line in raw_file:
            json_file.write(line.replace('}[', '}\n,'))

# Read json data and convert to pandas dataframe

df_bench = None

with open(bench_file_name + '_fixed.json') as json_file:
    
    data = json.load(json_file)
    
    df_bench = pd.DataFrame.from_records(data)

    df_bench[['id', 'n', 'dummy', 'epi', 'neu', 'rug', 'lambdaOne', 'lambda2', 'crossover', 'mutation', 'runtime', 'runtime over n']] = df_bench[['id', 'n', 'dummy', 'epi', 'neu', 'rug', 'lambdaOne', 'lambda2', 'crossover', 'mutation', 'runtime', 'runtime over n']].astype(float)

display(df_bench)

In [ ]:
# Count the mean runtimes of multiple runs

df_bench = df_bench.groupby(['id', 'n', 'dummy', 'epi', 'neu', 'lambdaOne', 'lambda2', 'crossover', 'mutation', 'rug', 'algorithm'])['runtime', 'runtime over n'].mean()
df_all = df_bench.reset_index()
display(df_all)

In [ ]:
# Convert values to float

df_all[['runtime', 'runtime over n']] = df_all[['runtime', 'runtime over n']].astype(float)

display(df_all)

In [ ]:
# Add group ranks for algorithms

df_all['id_rank'] = df_all.groupby('id')['runtime'].rank()

In [ ]:
# Runtime diff function

def _diff(x, algo_name):
    x['runtime'] = x['runtime'] - x[x['algorithm'] == algo_name]['runtime'].iloc[0]
    return x

In [ ]:
# Count runtime diffs

grouped = df_all.groupby('id')['algorithm', 'runtime']

df_all['diff_to_trueSB'] = grouped.apply(_diff, 'trueSB')['runtime']
df_all['diff_to_RLS'] = grouped.apply(_diff, 'RLS')['runtime']
df_all['diff_to_(1+1) EA'] = grouped.apply(_diff, '(1+1) EA')['runtime']
df_all['diff_to_(1+(λ,λ)) GA, λ<=2ln n'] = grouped.apply(_diff, '(1+(λ,λ)) GA, λ<=2ln n')['runtime']
df_all['diff_to_(1+(λ,λ)) GA, λ<=n'] = grouped.apply(_diff, '(1+(λ,λ)) GA, λ<=n')['runtime']
df_all['diff_to_(1+(λ,λ)) GA, λ=4'] = grouped.apply(_diff, '(1+(λ,λ)) GA, λ=4')['runtime']
df_all['diff_to_(1+(λ,λ)) GA, λ~pow(2.5)'] = grouped.apply(_diff, '(1+(λ,λ)) GA, λ~pow(2.5)')['runtime']

display(df_all)

In [ ]:
# Compute statistics

algorithms = df_all['algorithm'].unique()
values = {algo : {'algorithm': algo} for algo in algorithms}

for algo in values:
    values[algo]['avg_rank'] = df_all[df_all.algorithm == algo]['id_rank'].mean()
    values[algo]['std_rank'] = df_all[df_all.algorithm == algo]['id_rank'].std()
    values[algo]['avg_diff_to_trueSB'] = df_all[df_all.algorithm == algo]['diff_to_trueSB'].mean()
    values[algo]['std_diff_to_trueSB'] = df_all[df_all.algorithm == algo]['diff_to_trueSB'].std()
    values[algo]['avg_diff_to_RLS'] = df_all[df_all.algorithm == algo]['diff_to_RLS'].mean()
    values[algo]['std_diff_to_RLS'] = df_all[df_all.algorithm == algo]['diff_to_RLS'].std()
    values[algo]['avg_diff_to_(1+1) EA'] = df_all[df_all.algorithm == algo]['diff_to_(1+1) EA'].mean()
    values[algo]['std_diff_to_(1+1) EA'] = df_all[df_all.algorithm == algo]['diff_to_(1+1) EA'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ<=2ln n'] = df_all[df_all.algorithm == algo]['diff_to_(1+(λ,λ)) GA, λ<=2ln n'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ<=2ln n'] = df_all[df_all.algorithm == algo]['diff_to_(1+(λ,λ)) GA, λ<=2ln n'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ<=n'] = df_all[df_all.algorithm == algo]['diff_to_(1+(λ,λ)) GA, λ<=n'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ<=n'] = df_all[df_all.algorithm == algo]['diff_to_(1+(λ,λ)) GA, λ<=n'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ=4'] = df_all[df_all.algorithm == algo]['diff_to_(1+(λ,λ)) GA, λ=4'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ=4'] = df_all[df_all.algorithm == algo]['diff_to_(1+(λ,λ)) GA, λ=4'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ~pow(2.5)'] = df_all[df_all.algorithm == algo]['diff_to_(1+(λ,λ)) GA, λ~pow(2.5)'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ~pow(2.5)'] = df_all[df_all.algorithm == algo]['diff_to_(1+(λ,λ)) GA, λ~pow(2.5)'].std()
    
    # Neutrality

    values[algo]['avg_rank_neu_0_5'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 0) & (df_all.neu < 5)]['id_rank'].mean()
    values[algo]['std_rank_neu_0_5'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 0) & (df_all.neu < 5)]['id_rank'].std()
    values[algo]['avg_diff_to_trueSB _neu_0_5'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 0) & (df_all.neu < 5)]['diff_to_trueSB'].mean()
    values[algo]['std_diff_to_trueSB _neu_0_5'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 0) & (df_all.neu < 5)]['diff_to_trueSB'].std()
    values[algo]['avg_diff_to_RLS _neu_0_5'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 0) & (df_all.neu < 5)]['diff_to_RLS'].mean()
    values[algo]['std_diff_to_RLS _neu_0_5'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 0) & (df_all.neu < 5)]['diff_to_RLS'].std()
    values[algo]['avg_diff_to_(1+1) EA _neu_0_5'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 0) & (df_all.neu < 5)]['diff_to_(1+1) EA'].mean()
    values[algo]['std_diff_to_(1+1) EA _neu_0_5'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 0) & (df_all.neu < 5)]['diff_to_(1+1) EA'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ<=2ln n _neu_0_5'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 0) & (df_all.neu < 5)]['diff_to_(1+(λ,λ)) GA, λ<=2ln n'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ<=2ln n _neu_0_5'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 0) & (df_all.neu < 5)]['diff_to_(1+(λ,λ)) GA, λ<=2ln n'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ<=n _neu_0_5'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 0) & (df_all.neu < 5)]['diff_to_(1+(λ,λ)) GA, λ<=n'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ<=n _neu_0_5'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 0) & (df_all.neu < 5)]['diff_to_(1+(λ,λ)) GA, λ<=n'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ=4 _neu_0_5'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 0) & (df_all.neu < 5)]['diff_to_(1+(λ,λ)) GA, λ=4'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ=4 _neu_0_5'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 0) & (df_all.neu < 5)]['diff_to_(1+(λ,λ)) GA, λ=4'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ~pow(2.5) _neu_0_5'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 0) & (df_all.neu < 5)]['diff_to_(1+(λ,λ)) GA, λ~pow(2.5)'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ~pow(2.5) _neu_0_5'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 0) & (df_all.neu < 5)]['diff_to_(1+(λ,λ)) GA, λ~pow(2.5)'].std()

    values[algo]['avg_rank_neu_5_10'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 5) & (df_all.neu < 10)]['id_rank'].mean()
    values[algo]['std_rank_neu_5_10'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 5) & (df_all.neu < 10)]['id_rank'].std()
    values[algo]['avg_diff_to_trueSB _neu_5_10'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 5) & (df_all.neu < 10)]['diff_to_trueSB'].mean()
    values[algo]['std_diff_to_trueSB _neu_5_10'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 5) & (df_all.neu < 10)]['diff_to_trueSB'].std()
    values[algo]['avg_diff_to_RLS _neu_5_10'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 5) & (df_all.neu < 10)]['diff_to_RLS'].mean()
    values[algo]['std_diff_to_RLS _neu_5_10'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 5) & (df_all.neu < 10)]['diff_to_RLS'].std()
    values[algo]['avg_diff_to_(1+1) EA _neu_5_10'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 5) & (df_all.neu < 10)]['diff_to_(1+1) EA'].mean()
    values[algo]['std_diff_to_(1+1) EA _neu_5_10'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 5) & (df_all.neu < 10)]['diff_to_(1+1) EA'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ<=2ln n _neu_5_10'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 5) & (df_all.neu < 10)]['diff_to_(1+(λ,λ)) GA, λ<=2ln n'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ<=2ln n _neu_5_10'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 5) & (df_all.neu < 10)]['diff_to_(1+(λ,λ)) GA, λ<=2ln n'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ<=n _neu_5_10'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 5) & (df_all.neu < 10)]['diff_to_(1+(λ,λ)) GA, λ<=n'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ<=n _neu_5_10'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 5) & (df_all.neu < 10)]['diff_to_(1+(λ,λ)) GA, λ<=n'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ=4 _neu_5_10'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 5) & (df_all.neu < 10)]['diff_to_(1+(λ,λ)) GA, λ=4'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ=4 _neu_5_10'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 5) & (df_all.neu < 10)]['diff_to_(1+(λ,λ)) GA, λ=4'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ~pow(2.5) _neu_5_10'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 5) & (df_all.neu < 10)]['diff_to_(1+(λ,λ)) GA, λ~pow(2.5)'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ~pow(2.5) _neu_5_10'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 5) & (df_all.neu < 10)]['diff_to_(1+(λ,λ)) GA, λ~pow(2.5)'].std()
    
    values[algo]['avg_rank_neu_10_25'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 10) & (df_all.neu < 25)]['id_rank'].mean()
    values[algo]['std_rank_neu_10_25'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 10) & (df_all.neu < 25)]['id_rank'].std()
    values[algo]['avg_diff_to_trueSB _neu_10_25'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 10) & (df_all.neu < 25)]['diff_to_trueSB'].mean()
    values[algo]['std_diff_to_trueSB _neu_10_25'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 10) & (df_all.neu < 25)]['diff_to_trueSB'].std()
    values[algo]['avg_diff_to_RLS _neu_10_25'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 10) & (df_all.neu < 25)]['diff_to_RLS'].mean()
    values[algo]['std_diff_to_RLS _neu_10_25'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 10) & (df_all.neu < 25)]['diff_to_RLS'].std()
    values[algo]['avg_diff_to_(1+1) EA _neu_10_25'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 10) & (df_all.neu < 25)]['diff_to_(1+1) EA'].mean()
    values[algo]['std_diff_to_(1+1) EA _neu_10_25'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 10) & (df_all.neu < 25)]['diff_to_(1+1) EA'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ<=2ln n _neu_10_25'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 10) & (df_all.neu < 25)]['diff_to_(1+(λ,λ)) GA, λ<=2ln n'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ<=2ln n _neu_10_25'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 10) & (df_all.neu < 25)]['diff_to_(1+(λ,λ)) GA, λ<=2ln n'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ<=n _neu_10_25'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 10) & (df_all.neu < 25)]['diff_to_(1+(λ,λ)) GA, λ<=n'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ<=n _neu_10_25'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 10) & (df_all.neu < 25)]['diff_to_(1+(λ,λ)) GA, λ<=n'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ=4 _neu_10_25'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 10) & (df_all.neu < 25)]['diff_to_(1+(λ,λ)) GA, λ=4'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ=4 _neu_10_25'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 10) & (df_all.neu < 25)]['diff_to_(1+(λ,λ)) GA, λ=4'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ~pow(2.5) _neu_10_25'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 10) & (df_all.neu < 25)]['diff_to_(1+(λ,λ)) GA, λ~pow(2.5)'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ~pow(2.5) _neu_10_25'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 10) & (df_all.neu < 25)]['diff_to_(1+(λ,λ)) GA, λ~pow(2.5)'].std()
    
    values[algo]['avg_rank_neu_25_50'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25) & (df_all.neu < 50)]['id_rank'].mean()
    values[algo]['std_rank_neu_25_50'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25) & (df_all.neu < 50)]['id_rank'].std()
    values[algo]['avg_diff_to_trueSB _neu_25_50'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25) & (df_all.neu < 50)]['diff_to_trueSB'].mean()
    values[algo]['std_diff_to_trueSB _neu_25_50'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25) & (df_all.neu < 50)]['diff_to_trueSB'].std()
    values[algo]['avg_diff_to_RLS _neu_25_50'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25) & (df_all.neu < 50)]['diff_to_RLS'].mean()
    values[algo]['std_diff_to_RLS _neu_25_50'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25) & (df_all.neu < 50)]['diff_to_RLS'].std()
    values[algo]['avg_diff_to_(1+1) EA _neu_25_50'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25) & (df_all.neu < 50)]['diff_to_(1+1) EA'].mean()
    values[algo]['std_diff_to_(1+1) EA _neu_25_50'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25) & (df_all.neu < 50)]['diff_to_(1+1) EA'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ<=2ln n _neu_25_50'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25) & (df_all.neu < 50)]['diff_to_(1+(λ,λ)) GA, λ<=2ln n'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ<=2ln n _neu_25_50'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25) & (df_all.neu < 50)]['diff_to_(1+(λ,λ)) GA, λ<=2ln n'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ<=n _neu_25_50'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25) & (df_all.neu < 50)]['diff_to_(1+(λ,λ)) GA, λ<=n'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ<=n _neu_25_50'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25) & (df_all.neu < 50)]['diff_to_(1+(λ,λ)) GA, λ<=n'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ=4 _neu_25_50'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25) & (df_all.neu < 50)]['diff_to_(1+(λ,λ)) GA, λ=4'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ=4 _neu_25_50'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25) & (df_all.neu < 50)]['diff_to_(1+(λ,λ)) GA, λ=4'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ~pow(2.5) _neu_25_50'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25) & (df_all.neu < 50)]['diff_to_(1+(λ,λ)) GA, λ~pow(2.5)'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ~pow(2.5) _neu_25_50'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25) & (df_all.neu < 50)]['diff_to_(1+(λ,λ)) GA, λ~pow(2.5)'].std()
    
    values[algo]['avg_rank_neu_50_'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 50)]['id_rank'].mean()
    values[algo]['std_rank_neu_50_'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 50)]['id_rank'].std()
    values[algo]['avg_diff_to_trueSB _neu_50_'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25)]['diff_to_trueSB'].mean()
    values[algo]['std_diff_to_trueSB _neu_50_'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25)]['diff_to_trueSB'].std()
    values[algo]['avg_diff_to_RLS _neu_50_'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25)]['diff_to_RLS'].mean()
    values[algo]['std_diff_to_RLS _neu_50_'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25)]['diff_to_RLS'].std()
    values[algo]['avg_diff_to_(1+1) EA _neu_50_'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25)]['diff_to_(1+1) EA'].mean()
    values[algo]['std_diff_to_(1+1) EA _neu_50_'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25)]['diff_to_(1+1) EA'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ<=2ln n _neu_50_'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25)]['diff_to_(1+(λ,λ)) GA, λ<=2ln n'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ<=2ln n _neu_50_'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25)]['diff_to_(1+(λ,λ)) GA, λ<=2ln n'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ<=n _neu_50_'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25)]['diff_to_(1+(λ,λ)) GA, λ<=n'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ<=n _neu_50_'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25)]['diff_to_(1+(λ,λ)) GA, λ<=n'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ=4 _neu_50_'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25)]['diff_to_(1+(λ,λ)) GA, λ=4'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ=4 _neu_50_'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25)]['diff_to_(1+(λ,λ)) GA, λ=4'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ~pow(2.5) _neu_50_'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25)]['diff_to_(1+(λ,λ)) GA, λ~pow(2.5)'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ~pow(2.5) _neu_50_'] = df_all[(df_all.algorithm == algo) & (df_all.neu >= 25)]['diff_to_(1+(λ,λ)) GA, λ~pow(2.5)'].std()

    # Dummy
    
    values[algo]['avg_rank_dummy_095_1'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 1.0) & (df_all.dummy > 0.95)]['id_rank'].mean()
    values[algo]['std_rank_dummy_095_1'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 1.0) & (df_all.dummy > 0.95)]['id_rank'].std()
    values[algo]['avg_diff_to_trueSB _dummy_095_1'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 1.0) & (df_all.dummy > 0.95)]['diff_to_trueSB'].mean()
    values[algo]['std_diff_to_trueSB _dummy_095_1'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 1.0) & (df_all.dummy > 0.95)]['diff_to_trueSB'].std()
    values[algo]['avg_diff_to_RLS _dummy_095_1'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 1.0) & (df_all.dummy > 0.95)]['diff_to_RLS'].mean()
    values[algo]['std_diff_to_RLS _dummy_095_1'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 1.0) & (df_all.dummy > 0.95)]['diff_to_RLS'].std()
    values[algo]['avg_diff_to_(1+1) EA _dummy_095_1'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 1.0) & (df_all.dummy > 0.95)]['diff_to_(1+1) EA'].mean()
    values[algo]['std_diff_to_(1+1) EA _dummy_095_1'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 1.0) & (df_all.dummy > 0.95)]['diff_to_(1+1) EA'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ<=2ln n _dummy_095_1'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 1.0) & (df_all.dummy > 0.95)]['diff_to_(1+(λ,λ)) GA, λ<=2ln n'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ<=2ln n _dummy_095_1'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 1.0) & (df_all.dummy > 0.95)]['diff_to_(1+(λ,λ)) GA, λ<=2ln n'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ<=n _dummy_095_1'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 1.0) & (df_all.dummy > 0.95)]['diff_to_(1+(λ,λ)) GA, λ<=n'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ<=n _dummy_095_1'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 1.0) & (df_all.dummy > 0.95)]['diff_to_(1+(λ,λ)) GA, λ<=n'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ=4 _dummy_095_1'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 1.0) & (df_all.dummy > 0.95)]['diff_to_(1+(λ,λ)) GA, λ=4'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ=4 _dummy_095_1'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 1.0) & (df_all.dummy > 0.95)]['diff_to_(1+(λ,λ)) GA, λ=4'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ~pow(2.5) _dummy_095_1'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 1.0) & (df_all.dummy > 0.95)]['diff_to_(1+(λ,λ)) GA, λ~pow(2.5)'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ~pow(2.5) _dummy_095_1'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 1.0) & (df_all.dummy > 0.95)]['diff_to_(1+(λ,λ)) GA, λ~pow(2.5)'].std()
    
    values[algo]['avg_rank_dummy_090_095'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.95) & (df_all.dummy > 0.90)]['id_rank'].mean()
    values[algo]['std_rank_dummy_090_095'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.95) & (df_all.dummy > 0.90)]['id_rank'].std()
    values[algo]['avg_diff_to_trueSB _dummy_090_095'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.95) & (df_all.dummy > 0.90)]['diff_to_trueSB'].mean()
    values[algo]['std_diff_to_trueSB _dummy_090_095'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.95) & (df_all.dummy > 0.90)]['diff_to_trueSB'].std()
    values[algo]['avg_diff_to_RLS _dummy_090_095'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.95) & (df_all.dummy > 0.90)]['diff_to_RLS'].mean()
    values[algo]['std_diff_to_RLS _dummy_090_095'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.95) & (df_all.dummy > 0.90)]['diff_to_RLS'].std()
    values[algo]['avg_diff_to_(1+1) EA _dummy_090_095'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.95) & (df_all.dummy > 0.90)]['diff_to_(1+1) EA'].mean()
    values[algo]['std_diff_to_(1+1) EA _dummy_090_095'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.95) & (df_all.dummy > 0.90)]['diff_to_(1+1) EA'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ<=2ln n _dummy_090_095'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.95) & (df_all.dummy > 0.90)]['diff_to_(1+(λ,λ)) GA, λ<=2ln n'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ<=2ln n _dummy_090_095'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.95) & (df_all.dummy > 0.90)]['diff_to_(1+(λ,λ)) GA, λ<=2ln n'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ<=n _dummy_090_095'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.95) & (df_all.dummy > 0.90)]['diff_to_(1+(λ,λ)) GA, λ<=n'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ<=n _dummy_090_095'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.95) & (df_all.dummy > 0.90)]['diff_to_(1+(λ,λ)) GA, λ<=n'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ=4 _dummy_090_095'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.95) & (df_all.dummy > 0.90)]['diff_to_(1+(λ,λ)) GA, λ=4'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ=4 _dummy_090_095'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.95) & (df_all.dummy > 0.90)]['diff_to_(1+(λ,λ)) GA, λ=4'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ~pow(2.5) _dummy_090_095'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.95) & (df_all.dummy > 0.90)]['diff_to_(1+(λ,λ)) GA, λ~pow(2.5)'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ~pow(2.5) _dummy_090_095'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.95) & (df_all.dummy > 0.90)]['diff_to_(1+(λ,λ)) GA, λ~pow(2.5)'].std()
    
    values[algo]['avg_rank_dummy__090'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.90)]['id_rank'].mean()
    values[algo]['std_rank_dummy__090'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.90)]['id_rank'].std()
    values[algo]['avg_diff_to_trueSB _dummy__090'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.90)]['diff_to_trueSB'].mean()
    values[algo]['std_diff_to_trueSB _dummy__090'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.90)]['diff_to_trueSB'].std()
    values[algo]['avg_diff_to_RLS _dummy__090'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.90)]['diff_to_RLS'].mean()
    values[algo]['std_diff_to_RLS _dummy__090'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.90)]['diff_to_RLS'].std()
    values[algo]['avg_diff_to_(1+1) EA _dummy__090'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.90)]['diff_to_(1+1) EA'].mean()
    values[algo]['std_diff_to_(1+1) EA _dummy__090'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.90)]['diff_to_(1+1) EA'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ<=2ln n _dummy__090'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.90)]['diff_to_(1+(λ,λ)) GA, λ<=2ln n'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ<=2ln n _dummy__090'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.90)]['diff_to_(1+(λ,λ)) GA, λ<=2ln n'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ<=n _dummy__090'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.90)]['diff_to_(1+(λ,λ)) GA, λ<=n'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ<=n _dummy__090'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.90)]['diff_to_(1+(λ,λ)) GA, λ<=n'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ=4 _dummy__090'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.90)]['diff_to_(1+(λ,λ)) GA, λ=4'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ=4 _dummy__090'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.90)]['diff_to_(1+(λ,λ)) GA, λ=4'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ~pow(2.5) _dummy__090'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.90)]['diff_to_(1+(λ,λ)) GA, λ~pow(2.5)'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ~pow(2.5) _dummy__090'] = df_all[(df_all.algorithm == algo) & (df_all.dummy <= 0.90)]['diff_to_(1+(λ,λ)) GA, λ~pow(2.5)'].std()
    
    # N
    
    values[algo]['avg_rank_n_256'] = df_all[(df_all.algorithm == algo) & (df_all.n == 256)]['id_rank'].mean()
    values[algo]['std_rank_n_256'] = df_all[(df_all.algorithm == algo) & (df_all.n == 256)]['id_rank'].std()
    values[algo]['avg_diff_to_trueSB _n_256'] = df_all[(df_all.algorithm == algo) & (df_all.n == 256)]['diff_to_trueSB'].mean()
    values[algo]['std_diff_to_trueSB _n_256'] = df_all[(df_all.algorithm == algo) & (df_all.n == 256)]['diff_to_trueSB'].std()
    values[algo]['avg_diff_to_RLS _n_256'] = df_all[(df_all.algorithm == algo) & (df_all.n == 256)]['diff_to_RLS'].mean()
    values[algo]['std_diff_to_RLS _n_256'] = df_all[(df_all.algorithm == algo) & (df_all.n == 256)]['diff_to_RLS'].std()
    values[algo]['avg_diff_to_(1+1) EA _n_256'] = df_all[(df_all.algorithm == algo) & (df_all.n == 256)]['diff_to_(1+1) EA'].mean()
    values[algo]['std_diff_to_(1+1) EA _n_256'] = df_all[(df_all.algorithm == algo) & (df_all.n == 256)]['diff_to_(1+1) EA'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ<=2ln n _n_256'] = df_all[(df_all.algorithm == algo) & (df_all.n == 256)]['diff_to_(1+(λ,λ)) GA, λ<=2ln n'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ<=2ln n _n_256'] = df_all[(df_all.algorithm == algo) & (df_all.n == 256)]['diff_to_(1+(λ,λ)) GA, λ<=2ln n'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ<=n _n_256'] = df_all[(df_all.algorithm == algo) & (df_all.n == 256)]['diff_to_(1+(λ,λ)) GA, λ<=n'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ<=n _n_256'] = df_all[(df_all.algorithm == algo) & (df_all.n == 256)]['diff_to_(1+(λ,λ)) GA, λ<=n'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ=4 _n_256'] = df_all[(df_all.algorithm == algo) & (df_all.n == 256)]['diff_to_(1+(λ,λ)) GA, λ=4'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ=4 _n_256'] = df_all[(df_all.algorithm == algo) & (df_all.n == 256)]['diff_to_(1+(λ,λ)) GA, λ=4'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ~pow(2.5) _n_256'] = df_all[(df_all.algorithm == algo) & (df_all.n == 256)]['diff_to_(1+(λ,λ)) GA, λ~pow(2.5)'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ~pow(2.5) _n_256'] = df_all[(df_all.algorithm == algo) & (df_all.n == 256)]['diff_to_(1+(λ,λ)) GA, λ~pow(2.5)'].std()

    values[algo]['avg_rank_n_512'] = df_all[(df_all.algorithm == algo) & (df_all.n == 512)]['id_rank'].mean()
    values[algo]['std_rank_n_512'] = df_all[(df_all.algorithm == algo) & (df_all.n == 512)]['id_rank'].std()
    values[algo]['avg_diff_to_trueSB _n_512'] = df_all[(df_all.algorithm == algo) & (df_all.n == 512)]['diff_to_trueSB'].mean()
    values[algo]['std_diff_to_trueSB _n_512'] = df_all[(df_all.algorithm == algo) & (df_all.n == 512)]['diff_to_trueSB'].std()
    values[algo]['avg_diff_to_RLS _n_512'] = df_all[(df_all.algorithm == algo) & (df_all.n == 512)]['diff_to_RLS'].mean()
    values[algo]['std_diff_to_RLS _n_512'] = df_all[(df_all.algorithm == algo) & (df_all.n == 512)]['diff_to_RLS'].std()
    values[algo]['avg_diff_to_(1+1) EA _n_512'] = df_all[(df_all.algorithm == algo) & (df_all.n == 512)]['diff_to_(1+1) EA'].mean()
    values[algo]['std_diff_to_(1+1) EA _n_512'] = df_all[(df_all.algorithm == algo) & (df_all.n == 512)]['diff_to_(1+1) EA'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ<=2ln n _n_512'] = df_all[(df_all.algorithm == algo) & (df_all.n == 512)]['diff_to_(1+(λ,λ)) GA, λ<=2ln n'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ<=2ln n _n_512'] = df_all[(df_all.algorithm == algo) & (df_all.n == 512)]['diff_to_(1+(λ,λ)) GA, λ<=2ln n'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ<=n _n_512'] = df_all[(df_all.algorithm == algo) & (df_all.n == 512)]['diff_to_(1+(λ,λ)) GA, λ<=n'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ<=n _n_512'] = df_all[(df_all.algorithm == algo) & (df_all.n == 512)]['diff_to_(1+(λ,λ)) GA, λ<=n'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ=4 _n_512'] = df_all[(df_all.algorithm == algo) & (df_all.n == 512)]['diff_to_(1+(λ,λ)) GA, λ=4'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ=4 _n_512'] = df_all[(df_all.algorithm == algo) & (df_all.n == 512)]['diff_to_(1+(λ,λ)) GA, λ=4'].std()
    values[algo]['avg_diff_to_(1+(λ,λ)) GA, λ~pow(2.5) _n_512'] = df_all[(df_all.algorithm == algo) & (df_all.n == 512)]['diff_to_(1+(λ,λ)) GA, λ~pow(2.5)'].mean()
    values[algo]['std_diff_to_(1+(λ,λ)) GA, λ~pow(2.5) _n_512'] = df_all[(df_all.algorithm == algo) & (df_all.n == 512)]['diff_to_(1+(λ,λ)) GA, λ~pow(2.5)'].std()


In [ ]:
__algo = 'tuned'

for key in values[__algo]:
    if key.startswith('avg_rank'):
        print(key + ' : ' + str(values[__algo][key]))

In [ ]:
import matplotlib as plt
from matplotlib.pyplot import figure

In [ ]:
col = np.where(df_all.algorithm == 'RLS', 'yellow',
               np.where(df_all.algorithm == '(1+1) EA', 'orange',
                       np.where(df_all.algorithm == '(1+(λ,λ)) GA, λ<=2ln n', 'grey',
                               np.where(df_all.algorithm == '(1+(λ,λ)) GA, λ<=n', 'pink',
                                       np.where(df_all.algorithm == '(1+(λ,λ)) GA, λ=4', 'purple',
                                               np.where(df_all.algorithm == '(1+(λ,λ)) GA, λ~pow(2.5)', 'blue',
                                                       np.where(df_all.algorithm == 'trueSB', 'green',
                                                               np.where(df_all.algorithm == 'tuned', 'red', 'black'))))))))
p = df_all.plot(kind='scatter', x='id', y='runtime', color=col)
p.figure.set_size_inches(50, 50)
p.figure.savefig('try.png', dpi=200)